In [16]:
import pandas as pd
import numpy as np

# 데이터 불러오기
cp = pd.read_csv('/Users/user/Documents/DF/processed_cp.csv')


In [18]:
cp.head()

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,17,213000,Gasoline,a/t,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,22,143250,Gasoline,a/t,Gray,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,22,136731,Gasoline,a/t,Blue,Gray,None reported,Yes,13900
3,3,Genesis,7,19500,Gasoline,dct,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,3,7388,Gasoline,7-speed a/t,Black,Beige,None reported,Yes,97500


In [20]:
# 원핫인코딩할 열 지정
columns_to_encode = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# 원핫인코딩 수행
cp_encoded = pd.get_dummies(cp, columns=columns_to_encode, drop_first=True)

In [22]:
cp_y=cp_encoded['price']
cp_x=cp_encoded.drop(["id",'price'], axis=1)

In [24]:
# True/False를 1/0으로 변환
for column in cp_x.columns:
    cp_x[column] = cp_x[column].astype(int)

In [26]:
cp_x['milage'] = np.log1p(cp_x['milage'])
cp_y=np.log1p(cp_y)

In [42]:
cp_x.head()

,model_year,milage,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,brand_Bugatti,brand_Buick,brand_Cadillac,...,int_col_Etc,int_col_Gold,int_col_Gray,int_col_Green,int_col_Orange,int_col_Red,int_col_White,int_col_Yellow,accident_None reported,clean_title_Yes
0,17,12.269052,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
1,22,11.872354,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,22,11.825778,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
3,7,9.878221,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,3,8.907748,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [44]:
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트로 나누기 (여기서는 80% 훈련, 20% 테스트 비율)
X_train, X_test, y_train, y_test = train_test_split(cp_x, cp_y, test_size=0.2, random_state=42)


In [46]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 10g 값 변환 시 NaN 등의 이슈로 log()가 아닌 log1p()를 이용해 RMSLE 계산
def rmsle(y, pred):
    log_y = np.log1p(y)  # log1p(y) 사용
    log_pred = np.log1p(pred)  # log1p(pred) 사용
    squared_error = (log_y - log_pred) ** 2  # 제곱 오차 계산
    rmsle = np.sqrt(np.mean(squared_error))  # RMSLE 계산
    return rmsle

# 사이킷런의 mean_squared_error()를 이용해 RMSE 계산
def rmse(y, pred):
    return np.sqrt(mean_squared_error(y, pred))  # RMSE 계산

# MSE, RMSE, RMSLE를 모두 계산
def evaluate_regr(y, pred):
    rmsle_val = rmsle(y, pred)  # RMSLE 값 계산
    rmse_val = rmse(y, pred)  # RMSE 값 계산
    # MAE는 사이킷런의 mean_absolute_error()로 계산
    mae_val = mean_absolute_error(y, pred)  # MAE 값 계산
    print('RMSLE: {0:.3f}, RMSE: {1:.3f}, MAE: {2:.3f}'.format(rmsle_val, rmse_val, mae_val))  # 성능 평가 출력


In [48]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 선형 회귀 모델 생성
model = LinearRegression()

# 모델 훈련
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
pred = model.predict(X_test)





In [49]:
print(f"pred min: {np.min(pred)}, pred max: {np.max(pred)}")


pred min: 8.182748795277352, pred max: 12.703441641614575


In [50]:
y_test_exp=np.expm1(y_test)
pred_exp=np.expm1(pred)
evaluate_regr(y_test_exp,pred_exp)

RMSLE: 0.547, RMSE: 73491.911, MAE: 18672.931
